# Integrating Azure Key Vault with AKS

In this lesson, we will learn how to integrate Azure Key Vault with AKS clusters, leveraging *Managed Identity* for secure secrets retrieval, and modifying application code to enhance security.

## Managed Identity in Azure

> **Managed Identity** is a crucial security feature in Azure designed to simplify the authentication and authorization processes for applications and services. It eliminates the need for developers to manually manage credentials. Instead, it provides an automatically managed identity in Microsoft Entra ID for applications to use when connecting to resources that support Microsoft Entra ID authentication.

The primary purpose of Managed Identity is to provide a secure identity for applications, allowing them to connect to resources without the need for explicit credentials. It acts as a seamless and secure way for applications to obtain Microsoft Entra ID tokens, enabling secure communication without manual credential management.

Managed Identity is beneficial in scenarios such as deploying applications on Azure VMs, hosting web applications on Azure App Service, running containerized workloads on AKS, or using serverless architectures like Azure Functions. It simplifies authentication processes, enhances security, and eliminates the need for developers to manage explicit credentials. Additionally, it is particularly advantageous when applications need to securely retrieve secrets from Azure Key Vault.

### Managed Identity Types

Azure provides two types of managed identities:

1. **System-assigned Managed Identity**: When enabling a system-assigned managed identity:

   - Created directly on specific Azure resources (e.g., Virtual Machines or Azure App Service). Only the specified resource can use this managed identity to request tokens from Microsoft Entra ID.
   - A unique service principal is created in Microsoft Entra ID, tied to the Azure resource's lifecycle
   - The identity is automatically deleted when the associated Azure resource is deleted
   - Exclusive to the Azure resource it's created for, limiting sharing across resources <br><br>

2. **User-assigned Managed Identity**: When enabling an user-assigned managed identity:

   - Created independently as a standalone Azure resource
   - A dedicated service principal is established in Microsoft Entra ID, managed separately
   - This identity persists independently of the resources using it and must be explicitly deleted
   - Can be shared across multiple Azure resources, offering flexibility in identity management

System-assigned managed identities are ideal for single-resource workloads, where the identity is tied to that resource's lifecycle, whereas user-assigned managed identities are suited for multi-resource workloads requiring a shared identity. 

A common example of a multi-resource workload is a microservices architecture, where an application is decomposed into smaller, independently deployable services. Each service may run on a separate Azure resource, such as virtual machines, Azure Functions, or containerized instances. In this scenario, a user-assigned managed identity can be beneficial because it allows multiple services to share the same identity for secure access to shared resources, like databases or key vaults, while providing independent lifecycle management for the identity across these diverse resources. 

### Working with Managed Identities

When incorporating managed identities into your Azure resources, the process differs for user-assigned and system-assigned identities.

#### User-Assigned Managed Identity

To create a user-assigned managed identity, use the following command:

```bash 
# Create a user-assigned managed identity
az identity create -g myResourceGroup -n myUserAssignedIdentity
```

This command will create a new user-assigned managed identity named `myUserAssignedIdentity` within the specified resource group `myResourceGroup`. Upon successful execution, the command will return detailed information about the newly created user-assigned managed identity:

<p align=center> <img src=images/UserAssignedOutput.png width=500 height=250> </p>

After creating the user-assigned managed identity, you have two options: create a new resource with the identity or assign the identity to an existing resource. To create a resource with the user-assigned managed identity, use the following command:

```bash
az <resource> create --resource-group <myResourceGroup> --name <myResourceName> --assign-identity myUserAssignedIdentity --role contributor --scope <mySubscription>
```
This command will create a new Azure resource with the specified user-assigned managed identity. Adjust the `<resource>`, `<myResourceGroup>`,` <myResourceName>`, and `<mySubscription>` placeholders based on the resource type and your account details. Additional parameters may be required depending on the specific resource type you plan to create.

To assign the user-assigned managed identity to an existing resource use the following command:

```bash
az resource identity assign --resource-group myResourceGroup --name myResource --identities myUserAssignedIdentity
```

This command associates the user-assigned managed identity (`myUserAssignedIdentity`) with an existing resource (`myResource`) in the specified resource group (`myResourceGroup`).

#### System-Assigned Managed Identity

To enable a system-assigned managed identity during the creation of an Azure resource, use the following command:

```bash
# Enable system-assigned managed identity during the creation of an Azure resource
az <resource> create --resource-group <myResourceGroup> --name <myResourceName> --assign-identity --role contributor --scope <mySubscription>
```

To add system-assigned managed identity to an existing resource you can use the following command:

```bash
# Add system-assigned managed identity to an existing Azure resource
az <resource> identity assign --resource-group <myResourceGroup> --name <myResourceName>
```

To update an existing resource and enable system-assigned managed identity, use the following command:

```bash
# Update existing Azure resource to enable system-assigned managed identity
az <resource> update --resource-group <myResourceGroup> --name <myResourceName> --enable-managed-identity
```

> The first command (`az <resource> identity assign`) is specifically for assigning a system-assigned managed identity to a resource that didn't have it initially. The second command (`az <resource> update --enable-managed-identity`) enables the system-assigned managed identity feature for an existing resource. This means it activates the capability for the system to automatically create and manage the associated identity. The command itself doesn't explicitly assign a specific identity; instead, it allows the system to handle the identity creation and management automatically.

## Integrating Azure Key Vault with AKS

Integrating Azure Key Vault with AKS enhances security by allowing applications running in the AKS cluster to securely access sensitive information without exposing credentials. Setting this integration up includes enabling Managed Identity for AKS and assigning the necessary Key Vault permissions.

### Enabling Managed Identity for AKS

AKS provides a robust platform for deploying, managing, and scaling containerized applications using Kubernetes. Enabling Managed Identity for an AKS cluster enhances security and simplifies authentication, allowing applications within the cluster to securely connect with Azure services, such as Azure Key Vault and Azure SQL Database.

To enable Managed Identity for an AKS cluster, follow these steps:

1. Launch a command-line interface on your local machine. Sign in to your Azure account using the Azure CLI.

2. Use the following command to enable managed identity for an existing AKS cluster. Replace `<resource-group>` with the name of your AKS cluster's resource group and `<aks-cluster-name>` with your AKS cluster's name:

```bash
az aks update --resource-group <resource-group> --name <aks-cluster-name> --enable-managed-identity
```

3. Execute the following command to get information about the managed identity created for the AKS cluster:

```bash
az aks show --resource-group <resource-group> --name <aks-cluster-name> --query identityProfile
```

> Make a note of the `clientId` under `identityProfile` for later use.

Once Managed Identity is successfully enabled for your AKS cluster, you can leverage it for enhanced security and seamless interaction with various Azure services. In subsequent sections, we'll explore how to integrate Managed Identity into your AKS workloads, allowing applications within the cluster to securely access resources such as Azure Key Vault for managing secrets and other sensitive information. 

### Assigning Key Vault Permissions

**Role-Based Access Control (RBAC)** is a critical component of Azure's security, providing a structured approach to managing permissions within your cloud environment. RBAC ensures that users, services, and applications have precisely the required level of access, reducing the risk of unauthorized actions and enhancing overall security.

In the context of AKS, RBAC plays a vital role in controlling access to various resources. In this section, we will focus on assigning roles to the Managed Identity of an AKS cluster, granting it the necessary permissions to interact seamlessly and securely with Azure Key Vault.

At this point, if needed, it's beneficial to revisit the previous lesson, in which we covered different RBAC roles associated with Azure Key Vault.

In this example, we will assign the *Key Vault Secrets Officer* role to the Managed Identity linked with your AKS cluster. This role provides permissions to read, list, set and delete secrets, certificates, and keys within the specified Azure Key Vault. 

```bash
# Assign "Key Vault Secrets Officer" role to Managed Identity
az role assignment create --role "Key Vault Secrets Officer" \
  --assignee <managed-identity-client-id> \
  --scope /subscriptions/{subscription-id}/resourceGroups/{resource-group}/providers/Microsoft.KeyVault/vaults/{key-vault-name}
```

Replace `<managed-identity-client-id>` with the actual Managed Identity's client ID noted in the previous section, and update the scope to match your subscription, resource group, and Key Vault details.

By assigning this role, we ensure that the AKS cluster's Managed Identity has the permissions needed to access and manage secrets securely stored in Azure Key Vault.

> Once the integration is complete, applications deployed in your AKS cluster can securely retrieve secrets without exposing sensitive information. Managed Identity handles authentication, eliminating the need for explicit credentials.

## Azure Identity Libraries

**Azure Identity Libraries** provide a set of powerful tools for secure authentication and access management within Azure services. In Python, these libraries simplify the process of integrating identity-related functionalities into your applications, ensuring secure and seamless interactions with Azure resources.

Azure Identity Libraries in Python streamline the implementation of secure authentication mechanisms, offering features such as:

- **Managed Identity Integration**: Seamless integration with Azure Managed Identity, enabling applications to authenticate without the need for explicit credentials

- **Token Management**: Efficient token acquisition and management for Azure services, ensuring that applications always have up-to-date authentication tokens for secure access

- **Multi-Factor Authentication (MFA) Support**: Support for Multi-Factor Authentication, enhancing the security of your applications by adding an additional layer of verification

- **Microsoft Entra Directory Integration**: Integration with Microsoft Entra Directory for a centralized and secure identity management solution

### Modifying Python Application Code

In this section, we will walk through how to enhance security in Python applications, specifically the ones running in AKS clusters. By integrating it with Azure Identity and Azure Key Vault libraries, your AKS-based application can securely retrieve sensitive information, like credentials, from Azure Key Vault. This not only ensures robust security practices but also establishes a streamlined and reliable approach to managing and accessing your application's secrets within the context of AKS.

Before looking at the necessary code additions, ensure you have the following libraries installed:

```bash
pip install azure-identity
pip install azure-keyvault-secrets
```

Let's now look at an example of how you can modify a Python application code to retrieve secrets from Azure Key Vault using Managed Identity for AKS. 

```python
from azure.identity import ManagedIdentityCredential
from azure.keyvault.secrets import SecretClient

# Replace these values with your Key Vault details
key_vault_url = "https://your-key-vault-name.vault.azure.net/"

# Set up Azure Key Vault client with Managed Identity
credential = ManagedIdentityCredential()
secret_client = SecretClient(vault_url=key_vault_url, credential=credential)

# Access the secret values from Key Vault
secret = secret_client.get_secret("secret-name")

# Access the secret values
secret_value = secret.value

# Your application code can now use the retrieved secrets securely
```

Replace the placeholder values with the actual Key Vault details.

By incorporating the Azure Identity and Azure Key Vault libraries, coupled with the integration of Azure Key Vault and AKS set up above, your Python applications hosted on AKS gain the capability to access secrets stored in Azure Key Vault. This transformative approach replaces the need for hard-coding sensitive information within your application, introducing a more secure and dynamic credential management strategy. 

## Key Takeaways

- Azure Managed Identity eliminates the need for explicit credential management in your applications, providing a secure and seamless way to obtain Microsoft Identity tokens
- System-assigned managed identities are created directly on specific Azure resources, while user-assigned managed identities are standalone resources that can be shared across multiple Azure resources
- Assigning roles to managed identities plays a crucial role in regulating access to Azure resources
- Integrating AKS with Azure Key Vault facilitates secure storage and retrieval of sensitive information. Assigning roles to the AKS cluster's Managed Identity ensures proper permissions for accessing secrets.
- Leveraging Azure Identity and Key Vault libraries in Python applications running on AKS allows secure retrieval of secrets, reducing the reliance on hard-coded sensitive information
- The integration of Azure Identity and Key Vault libraries enables Python applications on AKS to access secrets securely, contributing to a more robust and secure operational workflow